In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Step 1: Load the CSV file (make sure the file has a column named 'SMILES')
df = pd.read_csv('path for smiles')

df

In [ ]:
import os
import pandas as pd
import subprocess
from multiprocessing import Pool, cpu_count

# Set output directory
output_dir = "path to output directory"
os.makedirs(output_dir, exist_ok=True)
#Generating 3-D structures
def smiles_to_3d(zincid, smiles):
    """Converts SMILES to 3D PDB using Open Babel."""
    log_messages = [f"Processing {zincid}..."]

    # Temporary files
    temp_smi = os.path.join(output_dir, f"{zincid}.smi")
    output_pdb = os.path.join(output_dir, f"{zincid}.pdb")

    try:
        # Save SMILES to a file
        with open(temp_smi, "w") as smi_file:
            smi_file.write(f"{smiles} {zincid}\n")

        # Run Open Babel with strict stereochemistry
        result = subprocess.run(
            ["obabel", temp_smi, "-O", output_pdb, "--gen3d", "--strictstereo"],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )

        # Check if the output file is valid
        if os.path.exists(output_pdb) and os.path.getsize(output_pdb) > 0:
            log_messages.append(f"Success: 3D structure saved as {output_pdb}")
        else:
            log_messages.append(f"Error: Empty PDB file for {zincid}!")

        os.remove(temp_smi)  # Clean up temp file

    except subprocess.CalledProcessError as e:
        log_messages.append(f"Open Babel error for {zincid}: {e}")

    return "\n".join(log_messages)

def process_row(row):
    """Processes a molecule using Open Babel."""
    return smiles_to_3d(row["ZINCID"], row["SMILES"])

def main():
    """Reads CSV and processes molecules in parallel."""
    print("Loading CSV file...")

    df = pd.read_csv('input file')
    rows = df.to_dict(orient="records") 

    print(f"Starting multiprocessing with {cpu_count()} cores.")

    with Pool(processes=cpu_count()) as pool:
        results = pool.map(process_row, rows)

    print("Processing completed:")
    for res in results:
        print(res)

if __name__ == "__main__":
    main()


In [ ]:
#Energy minimisation
import os
import pandas as pd
import subprocess
from multiprocessing import Pool, cpu_count

# Set input and output directories
input_dir = "path to directory containing 3D structure"
output_dir = "path where structures should be saved"  # Assuming output goes in the same directory
os.makedirs(output_dir, exist_ok=True)

def optimize_and_convert_pdb(zincid, pdb_file):
    """Optimizes the PDB structure and converts to PDBQT using Open Babel."""
    log_messages = [f"Processing {zincid}..."]

    # Paths for the optimized and PDBQT files
    optimized_pdb = os.path.join(output_dir, f"{zincid}_optimized.pdb")
    pdbqt_file = os.path.join(output_dir, f"{zincid}.pdbqt")

    try:
        # Step 1: Energy minimization of the PDB file
        result_min = subprocess.run(
            ["obabel", pdb_file, "-O", optimized_pdb, "--minimize"],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )

        if os.path.exists(optimized_pdb) and os.path.getsize(optimized_pdb) > 0:
            log_messages.append(f"Success: Structure optimized and saved as {optimized_pdb}")
        else:
            log_messages.append(f"Error: Minimization failed for {zincid}!")
            return "\n".join(log_messages)

        # Step 2: Convert the optimized PDB to PDBQT format (for AutoDock)
        result_pdbqt = subprocess.run(
            ["obabel", optimized_pdb, "-O", pdbqt_file],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )

        if os.path.exists(pdbqt_file) and os.path.getsize(pdbqt_file) > 0:
            log_messages.append(f"Success: Converted to PDBQT format and saved as {pdbqt_file}")
        else:
            log_messages.append(f"Error: Conversion to PDBQT failed for {zincid}!")

    except subprocess.CalledProcessError as e:
        log_messages.append(f"Error for {zincid}: {e}")

    return "\n".join(log_messages)

def process_row(row):
    """Processes a molecule by optimizing and converting the PDB."""
    pdb_file = os.path.join(input_dir, f"{row['ZINCID']}.pdb")
    return optimize_and_convert_pdb(row["ZINCID"], pdb_file)

def main():
    """Reads CSV and processes molecules in parallel."""
    print("Loading CSV file...")

    df = pd.read_csv("Structure.csv")
    rows = df.to_dict(orient="records")  # Process all rows, can limit this for testing

    print(f"Starting multiprocessing with {cpu_count()} cores.")

    with Pool(processes=cpu_count()) as pool:
        results = pool.map(process_row, rows)

    print("Processing completed:")
    for res in results:
        print(res)

if __name__ == "__main__":
    main()


In [ ]:
#Docking
import os
import subprocess
import pandas as pd
from multiprocessing import Pool, cpu_count

# Set directories
input_dir = "path where prepared ligands are stored"  # Directory with ligand pdbqt files
output_dir = "path to output"  # Assuming output goes here
receptor_pdbqt = "/home/saurav/Ibogaine/mormonomer.pdbqt"  # Fixed receptor file
os.makedirs(output_dir, exist_ok=True)
# Vina executable directory and executable name
vina_executable_dir = "/home/saurav/autodock_vina_1_1_2_linux_x86/bin"#specify where executable file is present
vina_executable = "./vina"  # No need for './' if it's in the current directory

# Change the working directory to where Vina is located
os.chdir(vina_executable_dir)

def extract_binding_energy(log_file):
    """Extract binding energy score from the Vina log file."""
    try:
        with open(log_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith("REMARK VINA RESULT"):
                    # Extract the binding energy from the line
                    score = float(line.split()[3])  # The binding energy is the 4th value
                    return score
    except Exception as e:
        print(f"Error extracting binding energy from {log_file}: {e}")
        return None

def run_vina(receptor_pdbqt, ligand_pdbqt, zincid):
    """Run AutoDock Vina docking."""
    log_file = os.path.join(output_dir, f"{zincid}_vina.txt")
    output_pdbqt = os.path.join(output_dir, f"{zincid}_vina_out.pdbqt")
    
    # AutoDock Vina command
    vina_command = [
        vina_executable,
        "--receptor", receptor_pdbqt,
        "--ligand", ligand_pdbqt,
        "--center_x", "139.974",  # Set the docking center coordinates
        "--center_y", "127.5",
        "--center_z", "126.64",
        "--size_x", "36",  # Set the docking grid size
        "--size_y", "40",
        "--size_z", "40",
        "--exhaustiveness", "100",  # Exhaustiveness of the docking search
        "--log", log_file,
        "--out", output_pdbqt
    ]
    
    try:
        # Run Vina command
        result = subprocess.run(
            vina_command,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        # Extract binding energy from the log file after docking
        binding_energy = extract_binding_energy(log_file)
        return zincid, binding_energy
    except subprocess.CalledProcessError as e:
        print(f"Error: Vina docking failed for {zincid}. Error: {e}")
        return zincid, None

def process_ligand(ligand_file):
    """Processes each ligand with the fixed receptor."""
    zincid = os.path.splitext(os.path.basename(ligand_file))[0]  # Extract ZINCID from ligand filename
    if os.path.exists(ligand_file):
        return run_vina(receptor_pdbqt, ligand_file, zincid)
    else:
        return zincid, None

def main():
    """Processes ligand files and stores binding energies in a DataFrame."""
    print("Searching for ligand PDBQT files in the directory...")

    # Get all ligand PDBQT files in the directory
    ligand_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".pdbqt")]
    
    if not ligand_files:
        print("No ligand PDBQT files found in the specified directory.")
        return

    print(f"Found {len(ligand_files)} ligand files. Starting multiprocessing with {cpu_count()} cores.")

    with Pool(processes=cpu_count()) as pool:
        results = pool.map(process_ligand, ligand_files)

    # Store the results in a DataFrame
    docking_results = []
    for zincid, binding_energy in results:
        docking_results.append({
            "ZINCID": zincid,
            "Binding Energy (kcal/mol)": binding_energy
        })

    df = pd.DataFrame(docking_results)
    print("Docking completed. Here are the results:")
    print(df)

    # Save the results to a CSV file
    df.to_csv(os.path.join(output_dir, "docking_results.csv"), index=False)

if __name__ == "__main__":
    main()
